<a href="https://colab.research.google.com/github/AminahMakhoul10/MT-ACID/blob/main/MT_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import zipfile
import pandas as pd

# Caminho do arquivo ZIP
zip_file_path = '/content/renaest_dabertos.zip'

# Descompactar o arquivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Carregar os arquivos CSV
acidentes = pd.read_csv('/content/Acidentes_DadosAbertos_20230912.csv', sep=';')
vitimas = pd.read_csv('/content/Vitimas_DadosAbertos_20230912.csv', sep=';')
tipoveiculo = pd.read_csv('/content/TipoVeiculo_DadosAbertos_20230912.csv', sep=';')
localidade = pd.read_csv('/content/Localidade_DadosAbertos_20230912.csv', sep=';')

# Verificar as colunas para garantir que estamos usando os nomes corretos
print("Colunas do DataFrame 'acidentes':", acidentes.columns)
print("Colunas do DataFrame 'vitimas':", vitimas.columns)
print("Colunas do DataFrame 'tipoveiculo':", tipoveiculo.columns)
print("Colunas do DataFrame 'localidade':", localidade.columns)


<ipython-input-35-e2c9adc743d1>:12: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  acidentes = pd.read_csv('/content/Acidentes_DadosAbertos_20230912.csv', sep=';')


Colunas do DataFrame 'acidentes': Index(['num_acidente', 'chv_localidade', 'data_acidente', 'uf_acidente',
       'ano_acidente', 'mes_acidente', 'mes_ano_acidente', 'codigo_ibge',
       'dia_semana', 'fase_dia', 'tp_acidente', 'cond_meteorologica',
       'end_acidente', 'num_end_acidente', 'cep_acidente', 'bairro_acidente',
       'km_via_acidente', 'latitude_acidente', 'longitude_acidente',
       'hora_acidente', 'tp_rodovia', 'cond_pista', 'tp_cruzamento',
       'tp_pavimento', 'tp_curva', 'lim_velocidade', 'tp_pista',
       'ind_guardrail', 'ind_cantcentral', 'ind_acostamento', 'qtde_acidente',
       'qtde_acid_com_obitos', 'qtde_envolvidos', 'qtde_feridosilesos',
       'qtde_obitos'],
      dtype='object')
Colunas do DataFrame 'vitimas': Index(['num_acidente', 'chv_localidade', 'data_acidente', 'uf_acidente',
       'ano_acidente', 'mes_acidente', 'mes_ano_acidente', 'faixa_idade',
       'genero', 'tp_envolvido', 'gravidade_lesao', 'equip_seguranca',
       'ind_motorista'

In [36]:
print(tipoveiculo.columns)


Index(['num_acidente', 'tipo_veiculo', 'ind_veic_estrangeiro',
       'qtde_veiculos'],
      dtype='object')


# Quais as cidades com mais acidentes?

In [37]:
# Filtrar dados para o estado do Mato Grosso
acidentes_mt = acidentes[acidentes['uf_acidente'] == 'MT']

# Contar o número de acidentes por cidade
acidentes_por_cidade_mt = acidentes_mt['chv_localidade'].value_counts().reset_index()
acidentes_por_cidade_mt.columns = ['chv_localidade', 'num_acidentes']

# Merge com o DataFrame de localidades para obter o nome da cidade
acidentes_por_cidade_mt = acidentes_por_cidade_mt.merge(localidade[['chv_localidade', 'municipio']], on='chv_localidade', how='left')

print(acidentes_por_cidade_mt[['municipio', 'num_acidentes']].head(10))



  municipio  num_acidentes
0    CUIABA            200
1    CUIABA            195
2    CUIABA            191
3    CUIABA            189
4    CUIABA            189
5    CUIABA            177
6    CUIABA            174
7    CUIABA            174
8    CUIABA            173
9    CUIABA            172


#  Quais as cidades com maior quantidade de acidentes por mil habitantes?

In [38]:
# Merge dos dados de acidentes com os dados de população
acidentes_populacao_mt = pd.merge(acidentes_por_cidade_mt, localidade[['chv_localidade', 'qtde_habitantes']], on='chv_localidade')

# Calcular acidentes por mil habitantes
acidentes_populacao_mt['acidentes_por_mil_habitantes'] = (acidentes_populacao_mt['num_acidentes'] / acidentes_populacao_mt['qtde_habitantes']) * 1000
acidentes_populacao_mt = acidentes_populacao_mt.sort_values(by='acidentes_por_mil_habitantes', ascending=False)

print(acidentes_populacao_mt[['municipio', 'acidentes_por_mil_habitantes']].head(10))


               municipio  acidentes_por_mil_habitantes
548           RIO BRANCO                      1.157631
899    PLANALTO DA SERRA                      1.129093
1618          ARAGUAINHA                      1.088139
304             CANARANA                      0.760694
163      BARRA DO GARCAS                      0.756082
683   PONTAL DO ARAGUAIA                      0.754262
598   PONTAL DO ARAGUAIA                      0.751541
166      BARRA DO GARCAS                      0.748379
165      BARRA DO GARCAS                      0.738722
544     BARAO DE MELGACO                      0.725602


# Quais os dias da semana com mais acidentes?

In [39]:

if 'dia_semana' in acidentes_mt.columns:
    # Contar o número de acidentes por dia da semana
    acidentes_por_dia_mt = acidentes_mt['dia_semana'].value_counts().reset_index()
    acidentes_por_dia_mt.columns = ['dia_semana', 'num_acidentes']
    print(acidentes_por_dia_mt)
else:
    print("A coluna 'dia_semana' não está presente no DataFrame de acidentes.")



      dia_semana  num_acidentes
0         SABADO           4412
1    SEXTA-FEIRA           3849
2        DOMINGO           3640
3   QUINTA-FEIRA           3453
4  SEGUNDA-FEIRA           3377
5   QUARTA-FEIRA           3365
6    TERCA-FEIRA           3299


#  Qual o percentual de acidentes causados por pessoas com embriaguez?

In [40]:
# Filtrar acidentes causados por embriaguez
acidentes_embriaguez_mt = acidentes_mt[acidentes_mt['tp_acidente'].str.contains('embriaguez', case=False, na=False)]
percentual_embriaguez_mt = (len(acidentes_embriaguez_mt) / len(acidentes_mt)) * 100
print(f'Percentual de acidentes causados por embriaguez: {percentual_embriaguez_mt:.2f}%')



Percentual de acidentes causados por embriaguez: 0.00%


# Quais são os números de acidentes por ano no estado do Mato Grosso?


In [41]:
# Contar o número de acidentes por ano no estado do Mato Grosso
acidentes_por_ano_mt = acidentes_mt['ano_acidente'].value_counts().reset_index()
acidentes_por_ano_mt.columns = ['ano_acidente', 'num_acidentes']
print(acidentes_por_ano_mt)


   ano_acidente  num_acidentes
0          2019           5431
1          2022           5333
2          2018           4833
3          2021           4821
4          2020           3777
5          2023           1200


# Quais são os tipos de acidentes mais comuns no estado do Mato Grosso:

In [42]:

if 'tp_acidente' in acidentes.columns:
    # Contar o número de acidentes por tipo de acidente
    acidentes_por_tipo_mt = acidentes_mt['tp_acidente'].value_counts().reset_index()
    acidentes_por_tipo_mt.columns = ['tp_acidente', 'num_acidentes']

    # Os 10 tipos de acidentes mais comuns
    print(acidentes_por_tipo_mt.head(10))
else:
    print("A coluna 'tp_acidente' não está presente no DataFrame de acidentes.")


                    tp_acidente  num_acidentes
0                       COLISAO          16714
1                  DESCONHECIDO           2183
2                         QUEDA           1874
3               COLISAO LATERAL            972
4              COLISAO TRASEIRA            827
5                        CHOQUE            796
6    ATROPELAMENTO COM PEDESTRE            723
7               COLISAO FRONTAL            662
8  OUTROS ACIDENTES DE TRANSITO            191
9                   CAPOTAMENTO            174


# Quais são os números de acidentes por tipo de veículo no estado do Mato Grosso?

In [43]:

if 'tipo_veiculo' in tipoveiculo.columns:
    # Merge dos dados de acidentes com o tipo de veículo
    acidentes_veiculo_mt = pd.merge(acidentes_mt, tipoveiculo[['num_acidente', 'tipo_veiculo']], on='num_acidente', how='left')

    # Contar o número de acidentes por tipo de veículo
    acidentes_por_tipo_veiculo_mt = acidentes_veiculo_mt['tipo_veiculo'].value_counts().reset_index()
    acidentes_por_tipo_veiculo_mt.columns = ['tipo_veiculo', 'num_acidentes']
    print(acidentes_por_tipo_veiculo_mt)
else:
    print("A coluna 'tipo_veiculo' não está presente no DataFrame de veículos.")


    tipo_veiculo  num_acidentes
0    MOTOCICLETA          10089
1      AUTOMOVEL           7065
2  NAO INFORMADO           2156
3   DESCONHECIDO           1734
4       CAMINHAO            891
5    CAMINHONETE            857
6      BICICLETA            548
7         ONIBUS            144
8         TRATOR              7
